# Приложение к ответу на тестовое задание для стажировки на проект "Пушкинская карта"

В рамках задания имеются следующие данные

1. о купленных билетах (идентификатор мероприятия; идентификатор участника; дата покупки; стоимость билета)
2. о возрасте и регионе участника программы (возраст — от 14 до 22, регион — один из 85 регионов РФ).
3. о мероприятиях в регионе посещения (тип организации; идентификатор мероприятия)
4. о факте посещения мероприятий (идентификатор мероприятия; идентификатор участника; дата посещения)

Для удобства мной были созданы (вручную) подобные датасеты со случайными данными

In [1]:
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

In [2]:
tickets = pd.read_csv('tickets.csv', encoding='koi8-r') # о купленных билетах
ages = pd.read_csv('ages.csv', encoding='koi8-r') # о возрасте и регионе участника программы
show = pd.read_csv('show.csv', encoding='koi8-r') # о мероприятиях в регионе посещения
visit = pd.read_csv('visit.csv', encoding='koi8-r') # о факте посещения мероприятий

### о купленных билетах

In [3]:
tickets.head()

,id_show,id_visitor,sell_date,ticket_cost
0,2,30,2020-12-06,890
1,3,45,2021-03-05,740
2,3,190,2021-03-24,780
3,1,83,2020-10-08,300
4,5,47,2021-06-09,150


In [4]:
tickets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id_show      9 non-null      int64 
 1   id_visitor   9 non-null      int64 
 2   sell_date    9 non-null      object
 3   ticket_cost  9 non-null      int64 
dtypes: int64(3), object(1)
memory usage: 416.0+ bytes


### о возрасте и регионе участника программы

In [5]:
ages.head()

,id_visitor,Age,Region
0,23,18,77
1,28,22,44
2,30,14,16
3,37,16,47
4,45,16,62


In [6]:
ages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   id_visitor  19 non-null     int64
 1   Age         19 non-null     int64
 2   Region      19 non-null     int64
dtypes: int64(3)
memory usage: 584.0 bytes


### о мероприятиях в регионе посещения

Предполагается что таблица show дана для одного региона и предсказания будут делаться для данного конкретного региона, а участники программы будут участвовать со всех регионов, ведь никто не запрещает (пока) приехать на мероприятие в другую область. Возможен и варинт общей таблицы, тогда надо конкатенировать таблицы по каждому региону с добавлением новой колонки с указанием региона проведения мероприятия.

In [7]:
show.head()

,kind_of_show,id_show
0,cinema,1
1,theatre,2
2,theatre,3
3,exhibition,4
4,theatre,5


In [8]:
show.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   kind_of_show  6 non-null      object
 1   id_show       6 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 224.0+ bytes


### о факте посещения мероприятий

In [9]:
visit.head()

,id_show,id_visitor,show_date
0,2,30,2020-12-10
1,3,45,2021-03-27
2,3,190,2021-03-27
3,1,83,2020-10-08
4,5,47,2021-06-12


In [10]:
visit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id_show     8 non-null      int64 
 1   id_visitor  8 non-null      int64 
 2   show_date   8 non-null      object
dtypes: int64(2), object(1)
memory usage: 320.0+ bytes


## EDA

Соберем все четыре наших датасета в один

In [11]:
# Объединим таблицы show и visit
join_one = show.merge(visit, on='id_show', how='outer')
join_one.head(20) 

,kind_of_show,id_show,id_visitor,show_date
0,cinema,1,83,2020-10-08
1,theatre,2,30,2020-12-10
2,theatre,3,45,2021-03-27
3,theatre,3,190,2021-03-27
4,exhibition,4,57,2021-05-28
5,theatre,5,47,2021-06-12
6,museum,6,33,2021-10-27
7,museum,6,118,2021-09-27


In [12]:
# - Сздадим список ID всех мероприятий
list_id_show = [] 
for cell in show['id_show']:
    list_id_show.append(cell)
print(list_id_show)        

[1, 2, 3, 4, 5, 6]


Создадим датафрейм где в каждой строке с данными участников присутствует ID мероприятия куда он мог бы пойти

In [13]:
ages['id_show'] = ages.apply(lambda x: list_id_show, axis=1) # Добавляем список с ID мероприятий для каждого участника
full_ages = ages.explode('id_show') # Преобразуем элементы списка в отдельные строки с этими элементами
full_ages.head(20)

,id_visitor,Age,Region,id_show
0,23,18,77,1
0,23,18,77,2
0,23,18,77,3
0,23,18,77,4
0,23,18,77,5
0,23,18,77,6
1,28,22,44,1
1,28,22,44,2
1,28,22,44,3
1,28,22,44,4


In [14]:
# объединим два предыдущих датасета
join_second = join_one.merge(full_ages, on=['id_show'], how='outer')
join_second.head(20) 

,kind_of_show,id_show,id_visitor_x,show_date,id_visitor_y,Age,Region
0,cinema,1,83,2020-10-08,23,18,77
1,cinema,1,83,2020-10-08,28,22,44
2,cinema,1,83,2020-10-08,30,14,16
3,cinema,1,83,2020-10-08,37,16,47
4,cinema,1,83,2020-10-08,45,16,62
5,cinema,1,83,2020-10-08,47,20,33
6,cinema,1,83,2020-10-08,33,14,76
7,cinema,1,83,2020-10-08,56,19,77
8,cinema,1,83,2020-10-08,61,21,77
9,cinema,1,83,2020-10-08,57,15,34


In [15]:
# Добавим в получившийся датасет данные из таблицы о билетах
full_join_data = tickets.merge(join_second, on=['id_show'], how='outer')
full_join_data.head(20) 

,id_show,id_visitor,sell_date,ticket_cost,kind_of_show,id_visitor_x,show_date,id_visitor_y,Age,Region
0,2,30,2020-12-06,890,theatre,30,2020-12-10,23,18,77
1,2,30,2020-12-06,890,theatre,30,2020-12-10,28,22,44
2,2,30,2020-12-06,890,theatre,30,2020-12-10,30,14,16
3,2,30,2020-12-06,890,theatre,30,2020-12-10,37,16,47
4,2,30,2020-12-06,890,theatre,30,2020-12-10,45,16,62
5,2,30,2020-12-06,890,theatre,30,2020-12-10,47,20,33
6,2,30,2020-12-06,890,theatre,30,2020-12-10,33,14,76
7,2,30,2020-12-06,890,theatre,30,2020-12-10,56,19,77
8,2,30,2020-12-06,890,theatre,30,2020-12-10,61,21,77
9,2,30,2020-12-06,890,theatre,30,2020-12-10,57,15,34


In [16]:
full_join_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 247 entries, 0 to 246
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id_show       247 non-null    object
 1   id_visitor    247 non-null    int64 
 2   sell_date     247 non-null    object
 3   ticket_cost   247 non-null    int64 
 4   kind_of_show  247 non-null    object
 5   id_visitor_x  247 non-null    int64 
 6   show_date     247 non-null    object
 7   id_visitor_y  247 non-null    int64 
 8   Age           247 non-null    int64 
 9   Region        247 non-null    int64 
dtypes: int64(6), object(4)
memory usage: 21.2+ KB


В этом датасете получилось аж три колонки с ID посетителя из которых:

1. id_visitor - номер человека пришедшего на шоу
2. id_visitor_x - номер человека купившего билет
3. id_visitor_y - номер человека зарегистрированного в системе

На первый взгляд полная бессмыслица, но основная мысль при таком объединении в том что если в одной строке все три ID совпадают, то это значит что человек зарегистрированный в системе купил билет на шоу из этой строки и посетил это шоу.

In [17]:
# Совпадение ID по строкам вычислю не самым элегантным способом: вычетом столбцов друг из друга
full_join_data['aux_col'] = 2*full_join_data['id_visitor'] - full_join_data['id_visitor_x'] - full_join_data['id_visitor_y']
full_join_data.head(20) 

,id_show,id_visitor,sell_date,ticket_cost,kind_of_show,id_visitor_x,show_date,id_visitor_y,Age,Region,aux_col
0,2,30,2020-12-06,890,theatre,30,2020-12-10,23,18,77,7
1,2,30,2020-12-06,890,theatre,30,2020-12-10,28,22,44,2
2,2,30,2020-12-06,890,theatre,30,2020-12-10,30,14,16,0
3,2,30,2020-12-06,890,theatre,30,2020-12-10,37,16,47,-7
4,2,30,2020-12-06,890,theatre,30,2020-12-10,45,16,62,-15
5,2,30,2020-12-06,890,theatre,30,2020-12-10,47,20,33,-17
6,2,30,2020-12-06,890,theatre,30,2020-12-10,33,14,76,-3
7,2,30,2020-12-06,890,theatre,30,2020-12-10,56,19,77,-26
8,2,30,2020-12-06,890,theatre,30,2020-12-10,61,21,77,-31
9,2,30,2020-12-06,890,theatre,30,2020-12-10,57,15,34,-27


Создадим столбец "result" который определяет пришел человек на шоу(1) или нет(0)

In [18]:
full_join_data['result'] = full_join_data['aux_col'].apply(lambda x: 1 if x==0 else 0)
full_join_data.tail(20) 

,id_show,id_visitor,sell_date,ticket_cost,kind_of_show,id_visitor_x,show_date,id_visitor_y,Age,Region,aux_col,result
227,4,61,2021-04-30,950,exhibition,57,2021-05-28,207,15,53,-142,0
228,4,57,2021-05-19,1000,exhibition,57,2021-05-28,23,18,77,34,0
229,4,57,2021-05-19,1000,exhibition,57,2021-05-28,28,22,44,29,0
230,4,57,2021-05-19,1000,exhibition,57,2021-05-28,30,14,16,27,0
231,4,57,2021-05-19,1000,exhibition,57,2021-05-28,37,16,47,20,0
232,4,57,2021-05-19,1000,exhibition,57,2021-05-28,45,16,62,12,0
233,4,57,2021-05-19,1000,exhibition,57,2021-05-28,47,20,33,10,0
234,4,57,2021-05-19,1000,exhibition,57,2021-05-28,33,14,76,24,0
235,4,57,2021-05-19,1000,exhibition,57,2021-05-28,56,19,77,1,0
236,4,57,2021-05-19,1000,exhibition,57,2021-05-28,61,21,77,-4,0


В таблице фактов посещения мероприятий было восемь записей. Таким образом в колонке "result" должно быть восемь значений (1)

In [19]:
full_join_data['result'].value_counts()

0    239
1      8
Name: result, dtype: int64

Проверка пройдена. Теперь избавляемся от ненужных признаков

In [20]:
data = full_join_data.drop(['id_show', 'id_visitor', 'id_visitor_x', 'id_visitor_y', 'aux_col', 'sell_date'], axis=1)
data.head()

,ticket_cost,kind_of_show,show_date,Age,Region,result
0,890,theatre,2020-12-10,18,77,0
1,890,theatre,2020-12-10,22,44,0
2,890,theatre,2020-12-10,14,16,1
3,890,theatre,2020-12-10,16,47,0
4,890,theatre,2020-12-10,16,62,0


Переведем значения в колонке "show_date" в формат datetime

In [21]:
# Функция для перевода в формат datetime
def str_to_date (a_d):
    real_date = datetime.strptime(a_d, '%Y-%m-%d')
    return real_date

# Собственно переводим значения в datetime
data['show_date'] = data['show_date'].apply(str_to_date)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 247 entries, 0 to 246
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ticket_cost   247 non-null    int64         
 1   kind_of_show  247 non-null    object        
 2   show_date     247 non-null    datetime64[ns]
 3   Age           247 non-null    int64         
 4   Region        247 non-null    int64         
 5   result        247 non-null    int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 13.5+ KB


Добавим пару признаков: день недели и месяц, т.к. для студентов, школотронов и для работающих людей это имеет значение.

In [22]:
data['month'] = data['show_date'].dt.month.astype('int16')
data['day_of_week'] = data['show_date'].dt.dayofweek.astype('int16')
data.sample()

,ticket_cost,kind_of_show,show_date,Age,Region,result,month,day_of_week
193,400,museum,2021-09-27,16,47,0,9,0


Теперь удалим столбец show_date, а так же перекодируем категориальные переменые с помощью getdummies (хотя энкодеры можно разные пробовать, но тут сделаем проще)

In [23]:
data = data.drop(['show_date'], axis=1)
data = pd.get_dummies(data, columns=[ 'kind_of_show', 'Region', 'month', 'day_of_week'])
data.head()

,ticket_cost,Age,result,kind_of_show_cinema,kind_of_show_exhibition,kind_of_show_museum,kind_of_show_theatre,Region_16,Region_33,Region_34,...,month_5,month_6,month_9,month_10,month_12,day_of_week_0,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5
0,890,18,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,890,22,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,890,14,1,0,0,0,1,1,0,0,...,0,0,0,0,1,0,0,1,0,0
3,890,16,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,1,0,0
4,890,16,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,1,0,0


Было бы не плохо обогатить данный датасет дополнительными данными, такими как: вид занятости участника(школьник, студент, безработный, работающий), пол участника, наличие романтических отношений, количество часов проводимых на работе/учебе.

Разобьем полученный датасет на тренировочный и валидационный

In [24]:
X = data.drop(['result'], axis=1) # Данные об участниках и мероприятиях
y = data['result'] # Таргет

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=82)

## Модели

В данном случае у нас задача классификации: пойдет человек на мероприятие (1) или нет(0). Поэтому выберем несколько соответствующих методов построения модели. В качестве метрики можно взять F1-score и ее состовляющие или ROC AUC. В рамках этого примера посмотрим на матрицу правдиво и ложно отрицательных-положительных результатов (confusion matrix), точность(precision), полноту(recall) и F1-score.

### Логистическая регрессия

In [25]:
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [26]:
Y_predicted = log_reg.predict(X_test)

print(confusion_matrix(y_test, Y_predicted))
print('f1_score:', f1_score(y_test, Y_predicted))
print('precision_score:', precision_score(y_test, Y_predicted))
print('recall_score:', recall_score(y_test, Y_predicted))

[[47  0]
 [ 3  0]]
f1_score: 0.0
precision_score: 0.0
recall_score: 0.0


C:\Users\Speedtriple\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Тут конечно ничего не получилось, потому что данных в примере маловато

### Решающие деревья

In [27]:
for_reg = RandomForestRegressor(n_estimators=100, random_state=82)    
for_reg.fit(X_train, y_train)  

RandomForestRegressor(random_state=82)

In [28]:
Y_predicted = log_reg.predict(X_test)

print(confusion_matrix(y_test, Y_predicted))
print('f1_score:', f1_score(y_test, Y_predicted))
print('precision_score:', precision_score(y_test, Y_predicted))
print('recall_score:', recall_score(y_test, Y_predicted))

[[47  0]
 [ 3  0]]
f1_score: 0.0
precision_score: 0.0
recall_score: 0.0


C:\Users\Speedtriple\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Результат тот же;)